In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

import warnings
warnings.simplefilter('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, GlobalAvgPool2D, AvgPool2D, MaxPool2D
from tensorflow.keras.layers import Flatten, Dense, Softmax, DepthwiseConv2D, BatchNormalization, ReLU
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [ ]:
df_train = pd.read_csv("../input/gtsrb-german-traffic-sign/Train.csv")
df_test = pd.read_csv("../input/gtsrb-german-traffic-sign/Test.csv")

Train_path = "../input/gtsrb-german-traffic-sign/Train"
Test_path = "../input/gtsrb-german-traffic-sign/Train"

In [ ]:
df_train.head()

In [ ]:
df_train.info()

## Ploting some Images

In [ ]:
df_img = df_train.sample(frac=1)
df_img = df_img.head()
df_img

In [ ]:
plt.figure(figsize=(16,9))
for i in range(5):
    plt.subplot(2,5,i+1)
    img = plt.imread("../input/gtsrb-german-traffic-sign/"+df_img.iloc[i][7])
    plt.title(df_img.iloc[i][6])
    plt.imshow(img)

In [ ]:
Total_classes = len(os.listdir(Train_path))
print("Total Traffic Signs: ",Total_classes)

In [ ]:
for i in os.listdir(Train_path):
    length = len(os.listdir(Train_path +'/'+ str(i)))
    print("Total No of Images in label {} are {} Images".format(i,length))

In [ ]:
df_train['ClassId'] = df_train['ClassId'].astype(str)
for i in range(0, len(df_train['ClassId'])):
    if len(df_train['ClassId'][i]) == 1:
        df_train['ClassId'][i] = '0' + df_train['ClassId'][i]

In [ ]:
df_test['ClassId'] = df_test['ClassId'].astype(str)
for i in range(0, len(df_test['ClassId'])):
    if len(df_test['ClassId'][i]) == 1:
        df_test['ClassId'][i] = '0' + df_test['ClassId'][i]

In [ ]:
train_gen = image.ImageDataGenerator(rescale=1./255,
                               vertical_flip=True,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               zoom_range=0.2)

testDataGen = image.ImageDataGenerator(rescale=1./255)

trainDataGenerator = train_gen.flow_from_dataframe(dataframe=df_train,
                                                   directory="../input/gtsrb-german-traffic-sign",
                                                   x_col='Path',
                                                   y_col='ClassId',
                                                   target_size=(299, 299),
                                                   batch_size=32,
                                                   class_mode='categorical')

testDataGenerator = testDataGen.flow_from_dataframe(dataframe=df_test,
                                                    directory='../input/gtsrb-german-traffic-sign/',
                                                    x_col='Path',
                                                    y_col='ClassId',
                                                    target_size=(299, 299),
                                                    batch_size=16,
                                                    class_mode='categorical')

In [ ]:
xception = Xception(include_top=False, weights=None, input_shape=(299, 299, 3), pooling='avg')
model = Sequential([
                    xception,
                    Dense(43,activation='softmax')
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(trainDataGenerator, epochs=5, verbose=1)

In [ ]:
#Training
plt.plot(history.history['accuracy'])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(['Train','Test'],loc='upper left')
plt.show()
#Loss
plt.plot(history.history['loss'])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(['Train','Test'],loc='upper left')
plt.show()

# Model Saving

In [ ]:
model.save("./saved_model/")

In [ ]:
model.save("Traffic_sign.h5")
loaded_model = load_model('Traffic_sign.h5', compile = False)

## Quantization

### Without quantization


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_model")
tflite_model = converter.convert()

In [ ]:
len(tflite_model)

### with quantization


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

In [ ]:
len(tflite_quant_model)

In [ ]:
with open("tflite_model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
with open("tflite_quant_model.tflite", "wb") as f:
    f.write(tflite_quant_model)

## Evaluating Model

In [ ]:
model.evaluate(testDataGenerator, verbose=1)

## Testing Model

In [ ]:
test_batch = 2
plt.figure(figsize=(64, 64))
plt.tight_layout()
for i in range(0, 16):
    plt.subplot(16, 2, 2 * i + 1), plt.axis('off')
    plt.imshow(testDataGenerator[test_batch][0][i])
    pre = model.predict_classes(testDataGenerator[test_batch][0])
    img = Image.open('../input/gtsrb-german-traffic-sign/Meta/' + str(pre[i]) + '.png')
    plt.subplot(16, 2, 2 * i + 2), plt.axis('off')
    plt.imshow(img)